In [ ]:
import numpy as np
import torch

<span style = 'font-size:1.2em;line-height:1.5em'><b>1. </b>실습 파일 "1_Activation_Function_Image_Read.ipynb"에 있는 numpy 기반의 activation함수들을 Pytorch기반의 코드로 변환하세요. 단, nn.ReLU(), F.relu()와 같이 Pytorch에서 기본으로 제공하는 활성화 함수를 사용하지 말고, 처음부터 짜보세요.</span>

In [ ]:
"""
예시
"""
def step_function_numpy(x):
    # x는 numpy array라고 가정
    y = x>0
    return y.astype(np.int64)

def step_function_torch(x):
    # x는 torch tensor라고 가정
    y = x>0
    return y.type(torch.int64)

## step_function_test
numpy_x = np.array([-1,3,5])
torch_x = torch.Tensor(numpy_x)
print(step_function_numpy(numpy_x))
print(step_function_torch(torch_x))

[0 1 1]
tensor([0, 1, 1])


In [ ]:
def sigmoid_numpy(x):
    """
    return sigmoid output
    """
    result = 1/(1+np.exp(-x))
    return result

def relu_numpy(x):
    """
    return ReLU output
    """
    result = np.maximum(x,0)
    return result

def softmax1_numpy(x):
    exp_sum = np.sum(np.exp(x))
    result = np.exp(x)/exp_sum
    return result

def softmax2_numpy(x):
    c=np.max(x)
    exp_x = np.exp(x-c) # overflow 대책
    exp_sum = np.sum(exp_x)
    result = exp_x/exp_sum
    return result


In [2]:
import torch

In [6]:
"""
아래 함수들을 작성하세요.
"""
def sigmoid_torch(x):
    """
    return sigmoid output
    """
    sigmoid_output = 1 / (1 + torch.exp(-x))
    return sigmoid_output


def relu_torch(x):
    """
    return ReLU output
    """
    ReLU_output = torch.relu(x)
    return ReLU_output

def softmax1_torch(x):
    """
    return softmax output
    """
    x = torch.tensor(x)
    exp_sum = torch.sum(torch.exp(x))
    softmax_output = torch.exp(x) / exp_sum
    return softmax_output

def softmax2_torch(x):
    """
    return softmax output with the trick
    """
    x = torch.tensor(x)  # NumPy 배열을 PyTorch 텐서로 변환
    c = torch.max(x)
    exp_x = torch.exp(x - c)  # overflow 대책
    exp_sum = torch.sum(exp_x)
    softmax_output_with_the_trick = exp_x / exp_sum
    return softmax_output_with_the_trick

In [ ]:
## step_function_test
x = np.array([-30,10,20])
print(sigmoid_numpy(x))
print(relu_numpy(x))
print(softmax1_numpy(x))
print(softmax2_numpy(x))


[9.35762297e-14 9.99954602e-01 9.99999998e-01]
[ 0 10 20]
[1.92866229e-22 4.53978687e-05 9.99954602e-01]
[1.92866229e-22 4.53978687e-05 9.99954602e-01]


In [ ]:
## step_function_test
x = torch.Tensor([-30,10,20])
print(sigmoid_torch(x))
print(relu_torch(x))
print(softmax1_torch(x))
print(softmax2_torch(x))


tensor([9.3576e-14, 9.9995e-01, 1.0000e+00])
tensor([ 0., 10., 20.])
tensor([1.9287e-22, 4.5398e-05, 9.9995e-01])
tensor([1.9287e-22, 4.5398e-05, 9.9995e-01])


<span style = 'font-size:1.2em;line-height:1.5em'><b>2. </b>이제까지 코드는 numpy 형식으로 되어있었습니다. 이를 Pytorch기반의 코드로 바꿔보세요. torch.nn.Linear()나 nn.ReLU()등의 High-level API를 사용하지 마시고, tensor 연산 기반의 코드로 low-level단에서 작성해보세요.</span>

- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 1. Activation function들을 Pytorch코드로 변환하셔야 됩니다. (1번문제의 결과물을 활용하셔도 되고, nn.ReLU()와 같은 pytorch에서 제공하는 함수를 사용하셔도 됩니다.)</span>
- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 2. network에 있는 numpy array를 Pytorch Tensor 형태로 변환하세요.</span>
- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 3. MNIST mini-batch data는 현재 numpy array인데 Pytorch Tensor 형태로 변환하세요.</span>
- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 4. Pytorch에서 텐서곱은 torch.matmul()입니다.</span>
- <span style = 'font-size:1.1em;line-height:1.3em'>Hint 5. np.argmax()와 torch.argmax()는 같은 역할을 한다.</span>

```python
a = torch.rand(size=(5,3))
print(a)
# 결과값
#tensor([[0.6298, 0.9776, 0.4705],
#        [0.4715, 0.6208, 0.1938],
#        [0.5101, 0.3516, 0.7683],
#        [0.5044, 0.5985, 0.1055],
#        [0.9975, 0.6862, 0.2044]])

torch.argmax(a, dim=1)
# 결과값
# tensor([1, 1, 2, 1, 0])
```

In [ ]:
import numpy as np
import torch
import pickle
from mnist import load_mnist
from act_func import sigmoid, softmax

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def init_network():
    fpath = 'sample_weights/sample_weight.pkl'
    with open(fpath, 'rb') as f:
        network = pickle.load(f)
    # network변수안의 모든 key에 대해서 torch.Tensor로 형태 변경
    # GPU사용할때를 위해서 해당 tensor를 device로 옮겨놓기

    return network

In [ ]:
def forward_propagation(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    # torch의 행렬곱과 activation function을 활용하여
    # forward propagation 나타내기

    return output

In [ ]:
(x_trn, y_trn), (x_tst, y_tst) = load_mnist(flatten=True, normalize=False)
network = init_network(device)

## 1. x_trn, y_trn을 device위에 있는 Tensor로 변환하고
## 2. forward_propgation을 진행한 결과를 results라는 변수에 저장한 뒤,
## 3. torch.argmax()함수를 사용하여 각 행마다 최대 값을 갖는 열의 index를 찾아낸다.
## 4. 그리고 위의 결과를 results라는 변수에다 저장한다.


print(f'미리 학습한 모델의 정확도는 {100*torch.sum(results==y_trn) / len(y_trn):.2f}%입니다.')


미리 학습한 모델의 정확도는 92.52%입니다.
